In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 2 layer
def neural_net(features) :
  layer1  = tf.sigmoid(tf.matmul(features,W1) + b1) # features : (1,2) W1 : (2,1), b1:(1,)
  layer2 = tf.sigmoid(tf.matmul(features,W2) + b2) # features : (1,2) W2: (2,1), b2:(1,)
  hypothesis = tf.sigmoid(tf.matmul(tf.concat([layer1,layer2],axis = -1),W3) + b3)  # tf. concat : (1,2) W3 : (2,1), b3:(1,)
  return hypothesis

In [5]:
# Vector
def neural_net2(features) :
  layer1  = tf.sigmoid(tf.matmul(features,W1) + b1)   # features shape : (4,2) W1 : (2,1) b1 : (2,)
  hypothesis = tf.sigmoid(tf.matmul(features,W2) + b2)
  return hypothesis

In [7]:
tf.random.set_seed(777)

In [8]:
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((x_data,y_data)).batch(len(x_data))

In [11]:
def preprocess_data(features, labels) :
  features = tf.cast(features, tf.float32)
  labels = tf.cast(labels, tf.float32)
  return features, labels

In [12]:
W1 = tf.Variable(tf.random.normal([2,1]), name = 'weight1')
b1 = tf.Variable(tf.random.normal([1]), name = 'bias1')

W2 = tf.Variable(tf.random.normal([2,1]), name = 'weight2')
b2 = tf.Variable(tf.random.normal([1]), name = 'bias2')

W3 = tf.Variable(tf.random.normal([2,1]), name = 'weight3')
b3 = tf.Variable(tf.random.normal([1]), name = 'bias3')


In [13]:
def neural_net_final(features) :
  layer1  = tf.sigmoid(tf.matmul(features,W1) + b1) 
  layer2 = tf.sigmoid(tf.matmul(features,W2) + b2) 
  layer3 = tf.concat([layer1,layer2],axis = -1) 
  layer3 = tf.reshape(layer3, shape = [-1,2])  # -1 : 자동
  hypothesis = tf.sigmoid(tf.matmul(layer3, W3) + b3)
  return hypothesis

In [27]:
def loss_fn(hypothesis, labels) :
  cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1-labels) * tf.math.log(1 - hypothesis))
  return cost

In [23]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)

In [24]:
def accuracy_fn(hypothesis, labels) :
  predicted = tf.cast(hypothesis> 0.5, dtype = tf.float32)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype = tf.float32))
  return accuracy

In [29]:
def grad(features, labels) :
  with tf.GradientTape() as tape :
    loss_value = loss_fn(neural_net_final(features),labels)
  return tape.gradient(loss_value, [W1,W2,W3,b1,b2,b3])

In [34]:
epochs = 10000
for step in range(epochs + 1) :
  for features, labels in iter(dataset) :
    features, labels = preprocess_data(features, labels)
    grads = grad(features, labels)
    optimizer.apply_gradients(grads_and_vars = zip(grads,[W1,W2,W3,b1,b2,b3]))
    if step % 500 == 0 :
      print("Iter : {}, Loss : {:.4f}".format(step, loss_fn(neural_net_final(features),labels)))



Iter : 0, Loss : 0.6847
Iter : 500, Loss : 0.6834
Iter : 1000, Loss : 0.6818
Iter : 1500, Loss : 0.6801
Iter : 2000, Loss : 0.6782
Iter : 2500, Loss : 0.6760
Iter : 3000, Loss : 0.6735
Iter : 3500, Loss : 0.6708
Iter : 4000, Loss : 0.6678
Iter : 4500, Loss : 0.6645
Iter : 5000, Loss : 0.6610
Iter : 5500, Loss : 0.6572
Iter : 6000, Loss : 0.6531
Iter : 6500, Loss : 0.6488
Iter : 7000, Loss : 0.6444
Iter : 7500, Loss : 0.6397
Iter : 8000, Loss : 0.6350
Iter : 8500, Loss : 0.6301
Iter : 9000, Loss : 0.6252
Iter : 9500, Loss : 0.6202
Iter : 10000, Loss : 0.6153


In [35]:
x_data, y_data = preprocess_data(x_data, y_data)
test_acc = accuracy_fn(neural_net(x_data),y_data)
print(test_acc)

tf.Tensor(0.75, shape=(), dtype=float32)
